In [ ]:
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 加载模型
model_name = "./models/qa_t5_epoch5"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 预测函数 
def predict(context, query, model, tokenizer, device="cpu", max_len=64, num_beams=4):
    input_text = f"问题: {query} 上下文: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_len,
            num_beams=num_beams,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 读取验证集数据 
with open("./data/DuReaderQG/dev.json", "r", encoding="utf-8") as f:
    dev_data = [json.loads(line) for line in f]

samples = dev_data[:10]  

# 预测
for i, sample in enumerate(samples, 1):
    context = sample["context"]
    question = sample["question"]
    true_answer = sample["answer"]

    pred_answer = predict(context, question, model, tokenizer, device)

    print(f"\n===== 样例 {i} =====")
    print("问题:", question)
    print("上下文:", context)
    print("真实答案:", true_answer)
    print("模型预测:", pred_answer)




===== 样例 1 =====
问题: 2017年银行贷款基准利率
上下文: 年基准利率4.35%。 从实际看,贷款的基本条件是: 一是中国大陆居民,年龄在60岁以下; 二是有稳定的住址和工作或经营地点; 三是有稳定的收入来源; 四是无不良信用记录,贷款用途不能作为炒股,赌博等行为; 五是具有完全民事行为能力。
真实答案: 年基准利率4.35%
模型预测: 4.35%

===== 样例 2 =====
问题: 2017年银行贷款基准利率
上下文: 年基准利率4.35%。 从实际看,贷款的基本条件是: 一是中国大陆居民,年龄在60岁以下; 二是有稳定的住址和工作或经营地点; 三是有稳定的收入来源; 四是无不良信用记录,贷款用途不能作为炒股,赌博等行为; 五是具有完全民事行为能力。
真实答案: 4.35%
模型预测: 4.35%

===== 样例 3 =====
问题: 格力空调哪个系列好
上下文: U系列是最好的，采用国际顶尖技术（由格力自主研发）双级变频压缩机，提高压缩机运转效率，制冷制热能力更强劲；1赫兹变频技术，使空调相当于一个15 W电灯泡，更加节能省电；送风面积广，风力大；生态风，净化空气。非常不错，现在国美在做活动，可以了解一下。
真实答案: U系列
模型预测: U系列

===== 样例 4 =====
问题: 橱柜宽度
上下文: 平面操作区域进深（即宽度）以40至60厘米为宜；要充分考虑洗菜盆的宽度。以标准洗菜盆来算，应选择550－－600MM的宽度为好。另：在高度方面，根据我国人体高度测算，掌握以下尺寸为宜：操作台高度在89至92厘米为宜；平面操作区域进深以40至60厘米为宜；抽油烟机与灶台的距离掌握在60至80厘米为宜；操作台上方的吊柜要能使主人操作时不碰头为宜，它距地面最小距离不应小于145厘米，进深尺寸为25至35厘米，吊柜与操作台之间的距离应在55厘米以上。
真实答案: 以40至60厘米为宜
模型预测: 550--600MM

===== 样例 5 =====
问题: 橱柜宽度
上下文: 平面操作区域进深（即宽度）以40至60厘米为宜；要充分考虑洗菜盆的宽度。以标准洗菜盆来算，应选择550－－600MM的宽度为好。另：在高度方面，根据我国人体高度测算，掌握以下尺寸为宜：操作台高度在89至92厘米为宜；平面操作区域进深以40至60厘米为